In [ ]:
## 
##%overwritefile
##%file:src1/test.py
##%stdout->:myfifo
## %cleartest
print("这是输入的名字8")
##test_begin
print("------测试代码------\n")
##test_end

In [ ]:
## 测试
##%overwritefile
##%file:src1/test3.py
##%smafterexec:myfifo R
##%smafterexec:myfifo R
##%smafterexec:myfifo R
##%smafterexec:myfifo R
##%smafterexec:myfifo R
##%smafterexec:myfifo R
##%log:0
##%cleartest
## %kcmd:list
# i=0
# while i<100:
print("这是输入的名字")
    # i+=1

##test_begin
print("------测试代码------\n")
##test_end

In [ ]:

import mmap
import contextlib
import time
import random

flag=b''
j=0
for i in range(1, 20):
  with contextlib.closing(mmap.mmap(-1, 4096, tagname='myfifo', access=mmap.ACCESS_DEFAULT)) as m:
    # for i in range(1, 20):
      ## 检查 4095
      m.seek(0)
      flag=m.read(1)
      if flag!=b'\x00':
        # print("wait...{0}".format(flag))
        time.sleep(1/1000)
        continue
      m.seek(1)
      m.write(("this is test{0}.\n".format(j)).encode())
      m.flush()
      j+=1
      ## 置flag=1
      m.seek(0)
      m.write(b'\x01')
      m.flush()
      time.sleep(1/1000)

In [ ]:
##%overwritefile
##%file:src1/test2.py
import multiprocessing
import os
import time
import random
def main2():
  # 1. 创建一个队列 (先进先出)
  q = multiprocessing.Queue(10) # 最多放10个数据。 如果不指定长度,默认最大(和硬件相关)
  # 2. 创建多个进程，将队列的引用当做实参进行传递
  p2 = multiprocessing.Process(target=_write, args=(q,))
  p2.start()

# 写数据进程执行的代码:
def _write(q):
    print('Process(%s) is writing...' % os.getpid())
    urls=["1","2","3","4","5","6","7","8","9","0",]
    for url in urls:
        q.put(url)
        print('Put %s to queue...' % url)
        time.sleep(random.random())
if __name__ == "__main__":
    # multiprocessing.freeze_support()
    main2() 
# main2()    

In [ ]:
import mmap
import contextlib
import time
import atexit
import platform
import tempfile
import os
from queue import Queue

fcntl = None
msvcrt = None
bLinux = True
if platform.system() != 'Windows':
    fcntl = __import__("fcntl")
    bLinux = True
else:
    msvcrt = __import__('msvcrt')
    bLinux = False
##
##
##
class CFileLock:
    def __init__(self, filename):
        # self.kobj=kobj
        # self.tmpdir = tempfile.TemporaryDirectory()
        self.tmpdir =tempfile.gettempdir()
        fname=filename+ '.lock'
        fname = os.path.join(self.tmpdir, fname)
        self.filename = fname
        self.file = None
    def __del__(self):
        pass
        # if os.path.exists(self.filename):
            # os.remove(self.filename)
        # self.tmpdir.cleanup()
    def lock(self):
        if bLinux is True:
            self.file = open(self.filename, 'wb')
            try:
                fcntl.flock(self.file, fcntl.LOCK_EX | fcntl.LOCK_NB)
            except:
                return False
        else:
            self.file = open(self.filename, 'wb')
            try:
                msvcrt.locking(self.file.fileno(), msvcrt.LK_LOCK, 1)
            except Exception as e:
                return False
        return True
 
    def unlock(self):
        try:
            if bLinux is True:
                fcntl.flock(self.file, fcntl.LOCK_UN)
                self.file.close()
            else:
                self.file.seek(0)
                msvcrt.locking(self.file.fileno(), msvcrt.LK_UNLCK, 1)
        except Exception as e:
            pass
def read_all_from_queue(queue):
    res = b''
    size = queue.qsize()
    while size != 0:
        res += queue.get_nowait()
        size -= 1
    return res
i=0
flock=CFileLock('myfifo')
_queue = Queue()

while i<1500:
# while True:
#   if flock.lock():
    s=b''
    flag=b''
    with contextlib.closing(mmap.mmap(-1, 4096, tagname='myfifo', access=mmap.ACCESS_DEFAULT)) as m:
        m.seek(0)
        flag=m.read(1)
        ## 检查flag是否为1，1为可读
        if flag!=b'\x01':
            # print("wait...")
            time.sleep(1/1000)
            continue
        m.seek(0)
        m.write(b'\x02')
        m.flush()
        m.seek(1)
        s = m.read()
        m.seek(0)
        m.write(b'\x00'*4096)
        m.flush()
        i+=1
        time.sleep(1/1000)
    s1=s.decode('UTF-8', errors='ignore').strip(b'\x00'.decode())
    if len(s1)>0:
        print(s1.strip()+str(i))
#   flock.unlock()
    time.sleep(1/1000)
#   _contents = read_all_from_queue(_queue)
#   s1=_contents.decode('UTF-8', errors='ignore').strip(b'\x00'.decode())
#   if len(s1)>0:
#     print(s1.strip())


In [4]:
## %sendrpcmsg:tcp://127.0.0.1:4252  stopsrv
## %sendrpcmsg:tcp://127.0.0.1:4252  stopsrv
##%srmafterexec:tcp://127.0.0.1:4255 中文消息\\n
print("代码结束后由 srmafterexec 发送rpcstdin 消息")

[MyPythonKernel092832] Info:The process :h:\Jupyter\ipynb\MyPython\tmpcz9fec59.py
[MyPythonKernel092832] Info:python3 h:\Jupyter\ipynb\MyPython\tmpcz9fec59.py 


代码结束后由 srmafterexec 发送rpcstdin 消息


[MyPythonKernel092832] Info:The process end:2312


In [ ]:
##%onlyruncmd
##%sendrpcmsg:tcp://127.0.0.1:4252  stopsrv

In [ ]:
import zerorpc

c = zerorpc.Client()
c.connect("ipc:///tmp/feeds/0")
print(c.hello("RPC"))

In [ ]:
print("这是关联文件代码的执行结果。😊")